In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

In [ ]:
eigenvalue_file_path = r"C:\Users\sulta\git\cone-operator-lab\mathematica\ellipsoid_eigs_a1_b1.5_c2.3_arnoldi-1000.txt"

eigs_all = np.loadtxt(eigenvalue_file_path, dtype=float)

eigs_index_range = (1, 400)  # Mathematica [1,400]
target_eigs = eigs_all[eigs_index_range[0]-1 : eigs_index_range[1]]
num_eigs = target_eigs.size

print("Subintervall:", eigs_index_range)
print("Genutzte Eigenwerte (N):", num_eigs)
print("Erster Eigenwert im Fit:", target_eigs[0])
print("Letzter Eigenwert im Fit:", target_eigs[-1])

In [ ]:
def fit_weyl_3_ls_reg(eigs, frac_range=(0.2, 0.9), lambda_reg=1e-6):
    """
    eigs: sortierte Liste/Array der Eigenwerte λ_k
    frac_range: (r1, r2) mit 0<r1<r2<=1
    lambda_reg: Regularisierung (Tikhonov) auf Einheitsmatrix
    """
    eigs = np.asarray(eigs, dtype=float)
    n = eigs.size

    r1, r2 = frac_range
    k_min = math.ceil(r1 * n)
    k_max = math.floor(r2 * n)

    # Spektralfenster auswählen (Mathematica: eigs[[kMin ;; kMax]])
    lam_vals = eigs[k_min-1 : k_max]
    k_vals = np.arange(k_min, k_max+1, dtype=float)

    # Designmatrix: Spalten = [λ^(3/2), λ, sqrt(λ)]
    X = np.column_stack([lam_vals**1.5, lam_vals, np.sqrt(lam_vals)])

    # Normalengleichungen
    ATA = X.T @ X          # 3x3
    ATy = X.T @ k_vals     # 3x1

    # Regularisierung λ * mean(diag(ATA)) * I
    mean_diag = ATA.diagonal().mean()
    reg = lambda_reg * mean_diag * np.eye(ATA.shape[0])

    coeffs = np.linalg.solve(ATA + reg, ATy)
    return coeffs  # [A0, A1, A2]

In [ ]:
fit_frac_range_main = (0.2, 0.9)
lambda_reg = 1e-6

A0fit, A1fit, A2fit = fit_weyl_3_ls_reg(
    target_eigenvalues,
    frac_range=fit_frac_range_main,
    lambda_reg=lambda_reg
)

vol_spec = 6.0 * np.pi**2 * A0fit

# wahres Volumen des Ellipsoids
trueA = 1.0
trueB = 1.5
trueC = 2.3
vol_true = 4.0 * np.pi / 3.0 * trueA * trueB * trueC

rel_error = abs(vol_spec - vol_true) / vol_true

print()
print("--- 3-Term-Weyl-Fit mit Regularisierung ---")
print("Fenster (k/N):", fit_frac_range_main)
print("A0 (LS, reg):", A0fit)
print("A1 (LS, reg):", A1fit)
print("A2 (LS, reg):", A2fit)
print("Volumen (aus Spektrum, 3-Term):", vol_spec)
print("Volumen (geometrisch):         ", vol_true)
print("Relativer Fehler:              ", rel_error)

In [ ]:
maxN = num_eigs

vol_conv_data = []
for m in range(2, maxN + 1):  # ab m=2
    eigs_sub = target_eigenvalues[:m]
    A0_sub, _, _ = fit_weyl_3_ls_reg(
        eigs_sub,
        frac_range=fit_frac_range_main,
        lambda_reg=lambda_reg
    )
    vol_sub = 6.0 * np.pi**2 * A0_sub
    vol_conv_data.append((m, vol_sub))

vol_conv_data = np.array(vol_conv_data)  # shape (maxN-1, 2)

plt.figure(figsize=(8, 4))
plt.plot(vol_conv_data[:, 0], vol_conv_data[:, 1], label="V_N aus Spektrum")
plt.axhline(vol_true, color="red", linestyle="--", label="V_true")
plt.xlabel("N (Anzahl erster Eigenwerte)")
plt.ylabel("Volumen-Schätzung V_N")
plt.legend()
plt.tight_layout()
plt.show()